In [1]:
import torch
import torch.nn as nn
#torch functional
import torch.nn.functional as F
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms


import numpy as np

%matplotlib inline

device = 'cpu'

In [2]:
class SimCLRLoss(nn.Module): #TODO check/test this loss
    def __init__(self, temperature=0.5):
        super(SimCLRLoss, self).__init__()
        self.temperature = temperature
        self.similarity_f = nn.CosineSimilarity(dim=2)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, zis, zjs):
        batch_size = zis.size(0)

        # Normalize the embeddings
        zis = F.normalize(zis, dim=1)
        zjs = F.normalize(zjs, dim=1)

        # Compute similarity
        representations = torch.cat([zis, zjs], dim=0)
        similarity_matrix = self.similarity_f(representations.unsqueeze(1), representations.unsqueeze(0))

        # Create the labels
        labels = torch.cat([torch.arange(batch_size) for i in range(2)], dim=0)
        labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()

        # Mask to remove positive samples from the similarity matrix
        mask = torch.eye(labels.shape[0], dtype=torch.bool)
        labels = labels[~mask].view(labels.shape[0], -1)
        similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)

        # Apply temperature
        similarity_matrix = similarity_matrix / self.temperature

        # Create target labels
        target = torch.arange(batch_size).to(device) #used to be .to(labels.device) and it DID not work! no idea why

        loss_i = self.criterion(similarity_matrix[:batch_size], target)
        loss_j = self.criterion(similarity_matrix[batch_size:], target)

        loss = (loss_i + loss_j) / 2
        return loss

criterion = SimCLRLoss(temperature=0.1)


#make random encoded 16 batch size 8
batch_size_curr = 8
latent_dim = 15

encoded_view1 = np.random.rand(batch_size_curr, latent_dim)
encoded_view2 = np.random.rand(batch_size_curr, latent_dim)

encoded_cat = np.concatenate((encoded_view1, encoded_view2), axis=0)

zis = encoded_cat[:batch_size_curr]
zjs = encoded_cat[batch_size_curr:]

loss_tf = criterion(torch.tensor(zis), torch.tensor(zjs))
print(loss_tf.item())



3.0429899411238557


In [5]:
#now calculate loss by hand using numpy

# Normalize the embeddings and calculate similarity
zis = zis / np.linalg.norm(zis, axis=1)[:, np.newaxis]
zjs = zjs / np.linalg.norm(zjs, axis=1)[:, np.newaxis]


representations = np.concatenate([zis, zjs], axis=0)
similarity_matrix = np.dot(representations, representations.T)

#create labels for cross entropy
labels = np.concatenate([np.arange(batch_size_curr) for i in range(2)], axis=0)
labels = (labels[:, np.newaxis] == labels[np.newaxis, :]).astype(np.float32)



mask = np.eye(labels.shape[0], dtype=bool)
labels = labels[~mask].reshape(labels.shape[0], -1)
similarity_matrix = similarity_matrix[~mask].reshape(similarity_matrix.shape[0], -1)

similarity_matrix /= 0.1

target = np.arange(batch_size_curr)


def cross_entropy(pred, target):
    pred = np.exp(pred)
    pred /= pred.sum(axis=1)[:, np.newaxis]
    return -np.log(pred[np.arange(len(pred)), target]).mean()
˝
loss_i = cross_entropy(similarity_matrix[:batch_size_curr], target)

loss_j = cross_entropy(similarity_matrix[batch_size_curr:], target)

loss = (loss_i + loss_j) / 2

print(loss)


print(loss_tf.item()/loss)

3.0429899411238557
1.0


In [11]:
target

array([0, 1, 2, 3, 4, 5, 6, 7])

In [7]:
similarity_matrix.shape

(16, 15)